In [ ]:
# 만든 그래프를 경위도 좌표계에 도시합니다.

In [ ]:
# create_graph에서 만든 그래프 자료를 로드합니다.

In [ ]:
import networkx as nx
import json
import pandas as pd

G_modified = nx.read_gml('../data/output/metro_graph.gml')

interchange_list = None
with open('../data/output/interchange_list.json', 'r') as f:
    interchange_list = json.load(f)

station_id_df = pd.read_parquet('../data/output/station_id_table.parquet')
id_no_dict, id_name_dict, _, id_x_dict, id_y_dict = station_id_df.set_index('station_id').T.to_dict('records')
id_coord_dict = {key: (id_x_dict[key], id_y_dict[key]) for key in id_x_dict.keys()}

line_no_df = pd.read_parquet('../data/output/line_no_table.parquet')
no_name_dict, no_color_dict = line_no_df.set_index('line_no').T.to_dict('records')

In [ ]:
# 그래프를 그립니다.
# 그래프 G는 환승역이 서로 간선으로 연결되어 있기 때문에 이를 식별하기 어렵습니다.
# 따라서 환승역 정점를 하나의 정점로 바꾸고(ID는 concatenating), 경위도는 평균점으로 합니다.

In [ ]:
for interchange_nodes in interchange_list:
    interchange_nodes.sort()
    
    # 하나의 정점으로 바꾼 역 ID, 노선, 역명입니다.
    new_station_id = ' '.join(interchange_nodes)
    new_station_name = id_name_dict[interchange_nodes[0]]
    new_line_no = ' '.join(map(id_no_dict.get, interchange_nodes))

    # 각 노드의 인접한 간선을 모두 구합니다.
    # 그래프에서 환승역 정점을 삭제합니다.
    adjacent_set = set()
    for interchange_node in interchange_nodes:
        adjacent_edges = ((key, value['line_no']) for key, value in G_modified[interchange_node].items() if value['line_no'] != '')
        adjacent_set.update(adjacent_edges) # 동일한 역 간에 연결된 간선(환승 간선)을 삭제합니다.
    G_modified.remove_nodes_from(interchange_nodes)

    # 환승역 정점의 평균 경위도를 구합니다.
    # 구한 경위도를 id_coord_dict에 추가합니다.
    xx, yy = zip(*map(id_coord_dict.get, interchange_nodes))
    mean_pos = (sum(xx)/len(xx), sum(yy)/len(yy))

    id_coord_dict[new_station_id] = mean_pos
    id_name_dict[new_station_id] = new_station_name

    # 환승역의 평균점 정점를 추가합니다.
    G_modified.add_node(new_station_id, line_no=new_line_no, station_name=new_station_name, pos=mean_pos, is_interchange=True)
    adjacent_set = set(x for x in adjacent_set if x[0] in G_modified.nodes())

    # 새로운 환승역과 인접한 정점의 간선을 다시 연결합니다.
    # 만일 adjacent_set에 같은 노드를 가르키는 요소가 존재할 경우, attrib의 line_no를 concatenate합니다.
    for adjacent_node, line_no in adjacent_set:
        if (new_station_id, adjacent_node) in G_modified.edges():
            G_modified[new_station_id][adjacent_node]['line_no'] += ' '+line_no
        else:
            G_modified.add_edge(new_station_id, adjacent_node, line_no=line_no)
    # concatenate된 line_no를 정렬합니다.
    G_modified[new_station_id][adjacent_node]['line_no'] = ' '.join(sorted(G_modified[new_station_id][adjacent_node]['line_no'].split()))

In [ ]:
# import geopandas as gpd

# ctprvn_gdf = gpd.read_file('../data/input/CTPRVN/CTPRVN.shp')
# sig_gdf = gpd.read_file('../data/input/SIG/SIG.shp')

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

plt.rcParams['figure.figsize'] = (10, 10)
plt.rcParams['font.family'] = 'Malgun Gothic'
_, ax = plt.subplots()

# 노선 도시 이전 행정 경계를 그립니다.
# ctprvn_gdf.plot(ax=ax, color='whitesmoke', edgecolor='lightgrey')
# sig_gdf.plot(ax=ax, color='whitesmoke', edgecolor='lightgrey')

# 간선(역간 연결)을 그립니다.
# 현재 4호선·수인분당선 한대앞역~오이도역 전 구간, 6호선·경의중앙선 공덕~효창공원 구간에서의 복선 도시가 이루어지지 않았습니다(20201222).
first_line_no = (no[0] for no in nx.get_edge_attributes(G_modified, 'line_no').values())
edge_colors = list(map(no_color_dict.get, first_line_no))
nx.draw(G_modified, pos=id_coord_dict, edge_color=edge_colors, ax=ax, node_size=0, width=1)

# 정점(역)을 그립니다.
interchange_pos = []
primary_pos = []

for station_id, attrib in G_modified.nodes(data=True):
    pos = attrib['pos']
    if attrib['is_interchange']:
        interchange_pos.append(pos)
    else:
        primary_pos.append(pos)

interchange_xx, interchange_yy = zip(*interchange_pos)
ax.scatter(interchange_xx, interchange_yy, s=12, c='r', linewidths=1, edgecolors='k', zorder=10)

xx, yy = zip(*primary_pos)
ax.scatter(xx, yy, s=8, c='w', linewidths=1, edgecolors='k', zorder=5)

# matplotlib 설정
ax.set_title('수도권 전철 노선도')
ax.set_axis_on()
ax.set_xlabel('longitude')
ax.set_ylabel('latitude')
# ax.set_xbound(126.3691303996574, 127.78836114785861) # 전체 노선도 기준
# ax.set_ybound(36.71067953528156, 38.00786035654195)
ax.set_xbound(126.74351841147657, 127.20476047389174) # 서울특별시 기준
ax.set_ybound(37.41463958768953, 37.71511317524018)
ax.set_aspect(109.958489129649955/88.74) # 1km 당 경도값을 1km당 위도값으로 나눈 것
ax.tick_params(left=True, bottom=True, labelleft=True, labelbottom=True)

# 범례 설정
legend_lines = (Line2D([0], [0], color=no_color_dict[line_no], lw=2) for line_no in no_name_dict.keys())
ax.legend(legend_lines, no_name_dict.values())

plt.savefig('../data/output/view_seoul.svg')
plt.show()